In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from glob import glob
from pathlib import Path

import cv2
import gc
import os

import warnings
warnings.filterwarnings('ignore')

In [2]:
root_dir = Path('/content/drive/MyDrive/brain_tumor_dataset')
classes = os.listdir(root_dir)
classes

['yes', 'no', 'Untitled0.ipynb']

In [3]:
brain_tumor = {
    'no' : list(root_dir.glob('no/*')),
    'yes' : list(root_dir.glob('yes/*'))
}

In [4]:
brain_tumor_label ={
    'no' : 0,
    'yes': 1
}

In [5]:
X, y = [], []
for tumor, images in brain_tumor.items():
    for image in images:
        img = cv2.imread(str(image))
        resized_img = cv2.resize(img, (180, 180))
        X.append(resized_img)
        y.append(brain_tumor_label[tumor])

In [6]:
X = np.array(X)
y = np.array(y)

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=43)

In [8]:
X_train_scaled = X_train / 255
X_test_scaled = X_test / 255

In [9]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import tensorflow as tf

In [10]:
from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomZoom
img_height = 180
img_width = 180
data_augmentation = Sequential(
    [
      RandomFlip("horizontal", input_shape=(img_height, img_width, 3)),
      RandomRotation(0.4),
      RandomZoom(0.4),
    ]
)

In [13]:
model = Sequential([
  data_augmentation,
  layers.Conv2D(1000, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(500, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(200, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(50, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(20, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(5, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.1),  # Adjust dropout rate
  layers.Flatten(),
  layers.Dense(180, activation='relu'),  # Reduce number of units
  layers.Dense(90, activation='relu'),
  layers.Dense(45, activation='relu'),
  layers.Dense(20, activation='relu'),
  layers.Dropout(0.1),  # Additional dropout layer
  layers.Dense(1, activation='sigmoid')  # Use softmax for multiclass classification
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),  # BinaryCrossentropy for binary classification
              metrics=['accuracy'])

model.fit(X_train_scaled, y_train, epochs=100,batch_size = 32)

Epoch 1/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 16s 2s/step - accuracy: 0.5426 - loss: 0.6924
Epoch 2/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.6444 - loss: 0.6831
Epoch 3/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.5920 - loss: 0.6677
Epoch 4/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - accuracy: 0.6189 - loss: 0.6479
Epoch 5/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.5989 - loss: 0.6475
Epoch 6/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.5827 - loss: 0.6758
Epoch 7/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.6054 - loss: 0.6479
Epoch 8/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - accuracy: 0.6042 - loss: 0.6601
Epoch 9/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.5775 - loss: 0.6797
Epoch 10/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.6137 - loss: 0.6524
Epoch 11/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - accuracy: 0.5827 - loss: 0.6618
Epoch 12/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.6506 - loss: 0.6181
E

In [14]:
model.evaluate(X_test_scaled,y_test)

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 360ms/step - accuracy: 0.8333 - loss: 0.4711


[0.49553877115249634, 0.8125]

In [15]:
y_predict = model.predict(X_test_scaled)
y_predict[:5]

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 410ms/step


array([[0.6032589 ],
       [0.2529996 ],
       [0.70404685],
       [0.6651093 ],
       [0.4904859 ]], dtype=float32)

In [16]:
y_pred = []
for element in y_predict:
    if element > 0.5:
        y_pred.append(1)
    else :
        y_pred.append(0)

In [17]:
from sklearn.metrics import confusion_matrix , classification_report

print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.77      0.77      0.77        26
           1       0.84      0.84      0.84        38

    accuracy                           0.81        64
   macro avg       0.81      0.81      0.81        64
weighted avg       0.81      0.81      0.81        64



In [22]:
model.save('brain_tumor_model.keras')

In [23]:
from google.colab import files

# Download the model
files.download('brain_tumor_model.keras')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>